# Install Dependencies
---

In [1]:
! pip install -r requirements.txt
! pip install 'stable-baselines3==1.8.0'
! pip install 'shimmy>=0.2.1'
! pip install matplotlib

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Import Dependencies
---

In [2]:
import os
import numpy as np
import pandas as pd
import shutil
from stable_baselines3.sac import SAC
from citylearn.agents.rbc import BasicRBC
from citylearn.citylearn import CityLearnEnv
from citylearn.data import DataSet
from citylearn.utilities import read_json, write_json
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

In [3]:
SCHEMA_TEMPLATE = 'citylearn_challenge_2022_phase_1'
TARGET_DATASET_DIRECTORY = 'ev_test_data'
SCHEMA_FILEPATH = os.path.join(TARGET_DATASET_DIRECTORY, SCHEMA_TEMPLATE, 'schema.json')

# Create Test Dataset
---

In [4]:
# Copy an existing dataset
if os.path.isdir(TARGET_DATASET_DIRECTORY):
    shutil.rmtree(TARGET_DATASET_DIRECTORY)
else:
    pass

DataSet.copy(SCHEMA_TEMPLATE, TARGET_DATASET_DIRECTORY)
schema = read_json(SCHEMA_FILEPATH)

# add ev to buildings
CAPACITITES = [100.0, 60.0, 80.0, 40.0]
NOMINAL_POWERS = [9.0, 10.0, 8.0, 12.0]
EV_COUNTS = [1, 2, 3, 4, 5]
ARRIVAL_HOUR = (7, 10)
DEPARTURE_HOUR = (15, 18)
ARRIVAL_SOC = (0.2, 0.6)
DAYS = 365
HOURS = list(range(1, 25))*DAYS
HOURS = pd.DataFrame(HOURS, columns=['hour'])
HOURS['day'] = (HOURS.index/HOURS['hour'].max()).astype(int)

for b in schema['buildings']:
    ev_count = np.random.choice(EV_COUNTS)
    evs = []

    for i in range(ev_count):
        # define object
        schedule_filename = f"{b}_EV_{int(i+1)}.csv"
        ev = {
            "type": "citylearn.energy_model.ElectricVehicle",
            "schedule": schedule_filename, 
            "autosize": False,
            "attributes": {
            "capacity": np.random.choice(CAPACITITES),
            "efficiency": 0.9,
            "capacity_loss_coefficient": 1e-05,
            "loss_coefficient": 0.0,
            "nominal_power": np.random.choice(NOMINAL_POWERS),
            "soc_minimum_limit": 0.1,
            "soc_maximum_limit": 0.8,
            "discharge_count_limit": 2
            }
        }
        evs.append(ev)

        # create schedule
        schedule = pd.DataFrame(columns=['arrival_hour', 'departure_hour', 'arrival_soc'])
        schedule['arrival_hour'] = np.random.choice(range(*ARRIVAL_HOUR), size=DAYS)
        schedule['departure_hour'] = np.random.choice(range(*DEPARTURE_HOUR), size=DAYS)
        schedule['arrival_soc'] = np.random.uniform(*ARRIVAL_SOC, size=DAYS)
        schedule['day'] = schedule.index
        schedule = schedule.merge(HOURS, on='day')
        schedule['availability'] = 0
        schedule.loc[(schedule['arrival_hour']<=schedule['hour']) & (schedule['departure_hour'] >= schedule['hour']), 'availability'] = 1
        schedule.loc[schedule['hour']!=schedule['arrival_hour'], 'arrival_soc'] = None
        schedule[['availability', 'arrival_soc']].to_csv(os.path.join(TARGET_DATASET_DIRECTORY, SCHEMA_TEMPLATE, schedule_filename), index=False)

    schema['buildings'][b]['electric_vehicles'] = evs

write_json(SCHEMA_FILEPATH, schema)

# Test Simulation
---

## RBC

In [5]:
env = CityLearnEnv(SCHEMA_FILEPATH)
model = BasicRBC(env)
model.learn(episodes=1)
display(env.evaluate())

,cost_function,value,name,level
0,annual_peak_average,1.368341,District,district
1,carbon_emissions_total,1.444529,District,district
2,cost_total,1.270829,District,district
3,daily_peak_average,1.574993,District,district
4,discomfort_delta_average,0.000000,District,district
...,...,...,...,...
59,discomfort_too_cold_proportion,NaN,Building_5,building
60,discomfort_too_hot_proportion,NaN,Building_5,building
61,discomfort_delta_minimum,0.000000,Building_5,building
62,discomfort_delta_maximum,0.000000,Building_5,building


## SB3 SAC

In [9]:
# train
episodes = 5
env = CityLearnEnv(SCHEMA_FILEPATH, central_agent=True)
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
model = SAC('MlpPolicy', env, learning_starts=env.unwrapped.time_steps)
model.learn(total_timesteps=episodes*model.learning_starts)

# evaluate
observations = env.reset()
done = False

while not env.done:
    print(f'\rTime step: {env.time_step + 1}/{env.time_steps}', end=' '*50)
    actions, _ = model.predict(observations, deterministic=True)
    observations, reward, done, _ = env.step(actions)

display(env.evaluate())

Time step: 8759/8760                                                  

,cost_function,value,name,level
0,annual_peak_average,1.022152,District,district
1,carbon_emissions_total,1.249431,District,district
2,cost_total,1.229843,District,district
3,daily_peak_average,1.064348,District,district
4,discomfort_delta_average,0.000000,District,district
...,...,...,...,...
59,discomfort_too_cold_proportion,NaN,Building_5,building
60,discomfort_too_hot_proportion,NaN,Building_5,building
61,discomfort_delta_minimum,0.000000,Building_5,building
62,discomfort_delta_maximum,0.000000,Building_5,building
